In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import font
from tkinter import messagebox as m_box
from tkinter import ttk

In [2]:
def countBlank(dataFrame_to_count_Blank):
    Columnnames=dataFrame_to_count_Blank.columns
    Column_name = []
    Blank_Counts = []
    for columns in Columnnames:
        Column_name.append(columns)
        Blank_Counts.append((dataFrame_to_count_Blank[columns] == '').sum())
    dataframe_with_Counts = pd.DataFrame()
    dataframe_with_Counts['column_Name']=Column_name
    dataframe_with_Counts['blank_Counts']=Blank_Counts
    return dataframe_with_Counts

In [3]:
def Main_F1():
    
    
    def NominatedNames_Explode(Raw_Data_def ,Index_Raw_data_Columns_def,i):
        optedOutRespondents = Raw_Data_def.loc[Raw_Data_def[Optout_Combined[(int(i)*2)+1]] == '1',"sender_Name"]
        Raw_Data_def = Raw_Data_def[Raw_Data_def[Optout_Combined[int(i)*2]] != '1']
                
        #print(Index_Raw_data_Columns_def)
        #print(Raw_Data_def.columns)
                
        Raw_Data_def = (Raw_Data_def.set_index(Index_Raw_data_Columns_def).stack().str.split(' ✘', expand=True).stack().unstack(-2).reset_index(-1, drop=True).reset_index())
                
        Raw_Data_def = Raw_Data_def[Raw_Data_def[qNames_Combined[i]] != '']
        for names in optedOutRespondents:
            Raw_Data_def = Raw_Data_def[Raw_Data_def[qNames_Combined[i]] != names]
            
        Final_Raw_Data = pd.merge(Raw_Data_def,
                                  RespondentDataDF_Label[Res_Col_Nominated],
                                  left_on=qNames_Combined[i],
                                  right_on=influencerName,
                                  how='left')
        Final_Raw_Data = Final_Raw_Data.drop_duplicates()
        #Final_Raw_Data.columns = [str(x).lower() for x in Final_Raw_Data.columns]
        return Final_Raw_Data
    
    
    
    def Aggregated_Data(Final_Raw_Data_def,i):        
        #Operation On Aggregate Data
        Res_Col_Nominated.insert(0,qNames_Combined[i])
        Aggregate_table = Final_Raw_Data_def[Res_Col_Nominated]
        Aggregate_table = Aggregate_table.fillna('')
        Aggregate_table_2 = Aggregate_table.groupby(Res_Col_Nominated).size().reset_index(name='Count')
        Res_Col_Nominated.remove(qNames_Combined[i])
        
        Agg_ColList = np
        Agg_ColList = Aggregate_table_2.columns
        Agg_Col_Names =[qNames_Combined[i],'Count']
        for colnames in Agg_ColList:
            if colnames.startswith('nominated_'):
                Agg_Col_Names.append(colnames)
        Aggregate_table_2 = Aggregate_table_2[Agg_Col_Names]
        Aggregate_table_2.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
        Aggregate_table_2 = Aggregate_table_2.sort_values('Count', ascending=False)
        #Aggregate_table_2 = Aggregate_table_2.replace('',np.nan)
        return Aggregate_table_2
    
    
    
    class nofileException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
    
    class NoInfluencerNameFound(Exception):
        def __init__(self,msg):
            super().__init__(msg)
            
    class TextboxBlankException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
            
    class DuplicateNameException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
    
    
    
    ## Validating Inputs
    
    try:
        if RawLabel_F1 !="" and RespLabel_F1 !="" and SchemaLabel_F1 !="":
            ProgressText = "All files Imported"
            ProgressLabel_F1['text'] = ProgressText
        else:
            raise nofileException("Please Provide entry in ALL the fields")
    
    except nofileException as e:
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    except Exception:
        ProgressText = "+ Please Provide entry in ALL the fields"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    

    
    ## Importing File
    ###      > Survey data With Label
    ###      > Survey Schema
    ###      > Respondent Data
    
    
    influencerName = textbox1_F1.get()
    File_Name = textbox2_F1.get()
    
    df = pd.read_excel(RawLabel_F1,dtype=object, keep_default_na = False)
    #SchemaDF = pd.read_csv(SchemaLabel_F1,sep="\t")
    SchemaDF = pd.read_csv(SchemaLabel_F1,sep="\t",dtype=object, na_filter = False)
    RespondentDataDF= pd.read_csv(RespLabel_F1,sep="\t")
    RespondentDataDF[influencerName] = RespondentDataDF[influencerName].str.strip()
    
    
    try:
        if influencerName in RespondentDataDF.columns:
            ProgressText = ProgressText + "\nInfluencer Name " +influencerName + " is Present in Respondent file"
            ProgressLabel_F1['text'] = ProgressText
        elif not influencerName.strip():
            raise TextboxBlankException("\nText Box cannot be left Blank")
            
        else:
            raise  NoInfluencerNameFound("\nInfluencer Name " + influencerName + " is NOT Present in Respondent file")
            
        
    except TextboxBlankException as e:
        ProgressText = ProgressText + "\nText Box Influencer Name cannot be left Blank"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', "Text Box Influencer Name cannot be left Blank")
        exit()
        return
        
            
    except NoInfluencerNameFound:
        ProgressText = ProgressText + "\nInfluencer Name " +influencerName + " is NOT Present in Respondent file"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', "\nInfluencer Name " +influencerName + " is NOT Present in Respondent file")
        exit()
        return
        
    except Exception:
        ProgressText = ProgressText + "\n + Influencer Name " +influencerName + " is NOT Present in Respondent file"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', "\n + Influencer Name " +influencerName + " is NOT Present in Respondent file")
        exit()
        return

        
    try:
        if not File_Name.strip():
            raise TextboxBlankException("\nText Box Survey Name cannot be left Blank")
        
    except TextboxBlankException:
        ProgressText = ProgressText + "\nText Box Survey Name cannot be left Blank"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', "Text Box Survey Name cannot be left Blank")
        exit()
        return
    except Exception:
        ProgressText = ProgressText + "\n + Text Box Survey Name cannot be left Blank"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', " + Text Box Survey Name cannot be left Blank")
        exit()
        return
    
    ##### Validating Names

    try:
        if RespondentDataDF[influencerName].duplicated().any():
            ProgressText = ProgressText + "\nInfluencer Name Column " +influencerName + " has duplicate names"
            ProgressLabel_F1['text'] = ProgressText
            raise DuplicateNameException("\nThere are duplicate Influencer Names in File")
    
    except DuplicateNameException as e:
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    except Exception:
        ProgressText = "+ There are duplicate Influencer Names in File"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
        
        
    #### Converting Respondent data into Labelled data

    ResColList = np
    ResColList = RespondentDataDF.columns.str.replace('demo_','nominated_demo_')
    RespondentDataDF.columns = ResColList

    Res_Col_Names =['respid',influencerName]
    Res_Col_Nominated =[influencerName]
    for colnames in ResColList:
        if colnames.startswith('nominated'):
            Res_Col_Names.append(colnames)
            Res_Col_Nominated.append(colnames)

    RespondentDataDF_Label = RespondentDataDF.copy()
    User_Label = dict(zip(SchemaDF['Answer Code'], SchemaDF['Answer Label']))

    for colnames in Res_Col_Names:
        if colnames.startswith('nominated'):
            RespondentDataDF_Label[colnames] = RespondentDataDF[colnames].map(User_Label)



    ## Operations With RAW Survey Data

    RawColList = np
    RawColList = df.columns.str.replace('demo_','sender_demo_')
    df.columns = RawColList

    ###Inserting Sender Name
    
    df.insert(8,'sender_Name','')
    
    Respondent_Name = dict(zip(RespondentDataDF['respid'], RespondentDataDF[influencerName]))
    df['sender_Name'] = df['respid'].map(Respondent_Name)

    All_cols = df.columns
    Raw_data_Columns = ['responseid','respid','status','Live_Test','Ind_Global','sender_Name','qhidTemplateToUse','l']

    Optout_Combined = []
    #qCodes_Combined = []
    qNames_Combined = []
    for colnames in All_cols:
        if colnames.startswith('sender_demo_') and colnames.count('_rr_')==0:
            Raw_data_Columns.append(colnames)
        if colnames.startswith('Optout'):
            Optout_Combined.append(str(colnames))
        if colnames.startswith('qNames'):
            qNames_Combined.append(str(colnames))
            

    extra_cols = {'sender_demo_StraightLiners','sender_demo_Speeder','sender_demo_SL'}
    Raw_data_Columns_after = [rem for rem in Raw_data_Columns if rem not in extra_cols]

    Raw_data_Columns_1 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_1 = Raw_data_Columns_after.copy()

    Raw_data_Columns_2 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_2 = Raw_data_Columns_after.copy()

    Raw_data_Columns_3 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_3 = Raw_data_Columns_after.copy()

    Raw_data_Columns_4 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_4 = Raw_data_Columns_after.copy()

    Raw_data_Columns_5 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_5 = Raw_data_Columns_after.copy()

    Raw_data_Columns_6 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_6 = Raw_data_Columns_after.copy()

    Raw_data_Columns_7 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_7 = Raw_data_Columns_after.copy()

    Raw_data_Columns_8 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_8 = Raw_data_Columns_after.copy()

    Raw_data_Columns_9 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_9 = Raw_data_Columns_after.copy()

    Raw_data_Columns_10  = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_10 = Raw_data_Columns_after.copy()





    OptLength = len(Optout_Combined)
    #print(OptLength)
    for i in range(OptLength):
        if i<2:
            Raw_data_Columns_1.append(Optout_Combined[i])
            Index_Raw_data_Columns_1.append(Optout_Combined[i])
        elif i>=2 and i<4:
            Raw_data_Columns_2.append(Optout_Combined[i])
            Index_Raw_data_Columns_2.append(Optout_Combined[i])
        elif i>=4 and i<6:
            Raw_data_Columns_3.append(Optout_Combined[i])
            Index_Raw_data_Columns_3.append(Optout_Combined[i])
        elif i>=6 and i<8:
            Raw_data_Columns_4.append(Optout_Combined[i])
            Index_Raw_data_Columns_4.append(Optout_Combined[i])
        elif i>=8 and i<10:
            Raw_data_Columns_5.append(Optout_Combined[i])
            Index_Raw_data_Columns_5.append(Optout_Combined[i])
        elif i>=10 and i<12:
            Raw_data_Columns_6.append(Optout_Combined[i])
            Index_Raw_data_Columns_6.append(Optout_Combined[i])
        elif i>=12 and i<14:
            Raw_data_Columns_7.append(Optout_Combined[i])
            Index_Raw_data_Columns_7.append(Optout_Combined[i])
        elif i>=14 and i<16:
            Raw_data_Columns_8.append(Optout_Combined[i])
            Index_Raw_data_Columns_8.append(Optout_Combined[i])
        elif i>=16 and i<18:
            Raw_data_Columns_9.append(Optout_Combined[i])
            Index_Raw_data_Columns_9.append(Optout_Combined[i])
        elif i>=18 and i<20:
            Raw_data_Columns_10.append(Optout_Combined[i])
            Index_Raw_data_Columns_10.append(Optout_Combined[i])

    ###Creating Excel File
    File_Save_Path = RawLabel_F1
    File_Save_Path = "/".join(File_Save_Path.split("/")[:-1])
    File_Save_Path = File_Save_Path+"/" +File_Name +"_Influencer_Report.xlsx"
    writer = pd.ExcelWriter(File_Save_Path, engine='xlsxwriter')

    File_Save_Path2 = RawLabel_F1
    File_Save_Path2 = "/".join(File_Save_Path2.split("/")[:-1])
    File_Save_Path2 = File_Save_Path2+"/" +File_Name +"_Check_Blanks.xlsx"
    writer2 = pd.ExcelWriter(File_Save_Path2, engine='xlsxwriter')
    
    
            
    qNamesLength = len(qNames_Combined)
    
    for i in range(qNamesLength):
        if i==0:
            #Index_Raw_data_Columns_1.append(qCodes_Combined[i])
            #Raw_data_Columns_1.append(qCodes_Combined[i])
            Raw_data_Columns_1.append(qNames_Combined[i])
            Raw_Data_1 = df[Raw_data_Columns_1]
            
            
            if Raw_Data_1[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_1 = NominatedNames_Explode(Raw_Data_1 ,Index_Raw_data_Columns_1,i)
                Aggregated_data_table_1 = Aggregated_Data(Final_Raw_Data_1,i)
                
                Final_Raw_Data_1 = Final_Raw_Data_1.drop([influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                Final_Raw_Data_1.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_1 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_1 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_1.columns = [str(x).lower() for x in Final_Raw_Data_1.columns]
                Aggregated_data_table_1.columns = [str(x).lower() for x in Aggregated_data_table_1.columns]
                
                Final_Raw_Data_1.to_excel(writer, sheet_name=Raw_data_Name_1, index = False)
                Aggregated_data_table_1.to_excel(writer, sheet_name=Aggregate_sheet_Name_1, index = False)
                
                Blank_Counts1 = countBlank(Final_Raw_Data_1)
                #Blank_Counts1.reset_index(inplace=True)
                Blank_Counts1.to_excel(writer2, sheet_name=Raw_data_Name_1, index = False)
            

        elif i==1:
            #Index_Raw_data_Columns_2.append(qCodes_Combined[i])
            #Raw_data_Columns_2.append(qCodes_Combined[i])
            Raw_data_Columns_2.append(qNames_Combined[i])
            Raw_Data_2 = df[Raw_data_Columns_2]
            
            if Raw_Data_2[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_2 = NominatedNames_Explode(Raw_Data_2,Index_Raw_data_Columns_2,i)
                Aggregated_data_table_2 = Aggregated_Data(Final_Raw_Data_2,i)
                
                Final_Raw_Data_2 = Final_Raw_Data_2.drop([influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                Final_Raw_Data_2.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_2 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_2 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_2.columns = [str(x).lower() for x in Final_Raw_Data_2.columns]
                Aggregated_data_table_2.columns = [str(x).lower() for x in Aggregated_data_table_2.columns]
                
                Final_Raw_Data_2.to_excel(writer, sheet_name=Raw_data_Name_2, index = False)
                Aggregated_data_table_2.to_excel(writer, sheet_name=Aggregate_sheet_Name_2, index = False)
                
                Blank_Counts2 = countBlank(Final_Raw_Data_2)
                #Blank_Counts2.reset_index(inplace=True)
                Blank_Counts2.to_excel(writer2, sheet_name=Raw_data_Name_2, index = False)
            
           
        elif i==2:
            #Index_Raw_data_Columns_3.append(qCodes_Combined[i])
            #Raw_data_Columns_3.append(qCodes_Combined[i])
            Raw_data_Columns_3.append(qNames_Combined[i])
            Raw_Data_3 = df[Raw_data_Columns_3]
            
            if Raw_Data_3[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_3 = NominatedNames_Explode(Raw_Data_3,Index_Raw_data_Columns_3,i)
                Aggregated_data_table_3 = Aggregated_Data(Final_Raw_Data_3,i)
                
                Final_Raw_Data_3 = Final_Raw_Data_3.drop([influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                Final_Raw_Data_3.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_3 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_3 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_3.columns = [str(x).lower() for x in Final_Raw_Data_3.columns]
                Aggregated_data_table_3.columns = [str(x).lower() for x in Aggregated_data_table_3.columns]
                
                Final_Raw_Data_3.to_excel(writer, sheet_name=Raw_data_Name_3, index = False)
                Aggregated_data_table_3.to_excel(writer, sheet_name=Aggregate_sheet_Name_3, index = False)
                
                Blank_Counts3 = countBlank(Final_Raw_Data_3)
                #Blank_Counts3.reset_index(inplace=True)
                Blank_Counts3.to_excel(writer2, sheet_name=Raw_data_Name_3, index = False)
            
            
        elif i==3:
            #Index_Raw_data_Columns_4.append(qCodes_Combined[i])
            #Raw_data_Columns_4.append(qCodes_Combined[i])
            Raw_data_Columns_4.append(qNames_Combined[i])
            Raw_Data_4 = df[Raw_data_Columns_4]
            
            if Raw_Data_4[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_4 = NominatedNames_Explode(Raw_Data_4,Index_Raw_data_Columns_4,i)
                Aggregated_data_table_4 = Aggregated_Data(Final_Raw_Data_4,i)
                
                Final_Raw_Data_4 = Final_Raw_Data_4.drop([influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                Final_Raw_Data_4.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_4 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_4 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_4.columns = [str(x).lower() for x in Final_Raw_Data_4.columns]
                Aggregated_data_table_4.columns = [str(x).lower() for x in Aggregated_data_table_4.columns]
                
                Final_Raw_Data_4.to_excel(writer, sheet_name=Raw_data_Name_4, index = False)
                Aggregated_data_table_4.to_excel(writer, sheet_name=Aggregate_sheet_Name_4, index = False)
                
                Blank_Counts4 = countBlank(Final_Raw_Data_4)
                #Blank_Counts4.reset_index(inplace=True)
                Blank_Counts4.to_excel(writer2, sheet_name=Raw_data_Name_4, index = False)
            
        elif i==4:
            #Index_Raw_data_Columns_5.append(qCodes_Combined[i])
            #Raw_data_Columns_5.append(qCodes_Combined[i])
            Raw_data_Columns_5.append(qNames_Combined[i])
            Raw_Data_5 = df[Raw_data_Columns_5]
            
            if Raw_Data_5[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_5 = NominatedNames_Explode(Raw_Data_5,Index_Raw_data_Columns_5,i)
                Aggregated_data_table_5 = Aggregated_Data(Final_Raw_Data_5,i)
                
                Final_Raw_Data_5 = Final_Raw_Data_5.drop([influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                Final_Raw_Data_5.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_5 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_5 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_5.columns = [str(x).lower() for x in Final_Raw_Data_5.columns]
                Aggregated_data_table_5.columns = [str(x).lower() for x in Aggregated_data_table_5.columns]
                
                Final_Raw_Data_5.to_excel(writer, sheet_name=Raw_data_Name_5, index = False)
                Aggregated_data_table_5.to_excel(writer, sheet_name=Aggregate_sheet_Name_5, index = False)
                
                Blank_Counts5 = countBlank(Final_Raw_Data_5)
                #Blank_Counts5.reset_index(inplace=True)
                Blank_Counts5.to_excel(writer2, sheet_name=Raw_data_Name_5, index = False)
            
            
        elif i==5:
            #Index_Raw_data_Columns_6.append(qCodes_Combined[i])
            #Raw_data_Columns_6.append(qCodes_Combined[i])
            Raw_data_Columns_6.append(qNames_Combined[i])
            Raw_Data_6 = df[Raw_data_Columns_6]
            
            if Raw_Data_6[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_6 = NominatedNames_Explode(Raw_Data_6,Index_Raw_data_Columns_6,i)
                Aggregated_data_table_6 = Aggregated_Data(Final_Raw_Data_6,i)
                
                Final_Raw_Data_6 = Final_Raw_Data_6.drop([influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                Final_Raw_Data_6.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_6 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_6 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_6.columns = [str(x).lower() for x in Final_Raw_Data_6.columns]
                Aggregated_data_table_6.columns = [str(x).lower() for x in Aggregated_data_table_6.columns]
                
                Final_Raw_Data_6.to_excel(writer, sheet_name=Raw_data_Name_6, index = False)
                Aggregated_data_table_6.to_excel(writer, sheet_name=Aggregate_sheet_Name_6, index = False)
                
                Blank_Counts6 = countBlank(Final_Raw_Data_6)
                #Blank_Counts6.reset_index(inplace=True)
                Blank_Counts6.to_excel(writer2, sheet_name=Raw_data_Name_6, index = False)
            
        elif i==6:
            #Index_Raw_data_Columns_7.append(qCodes_Combined[i])
            #Raw_data_Columns_7.append(qCodes_Combined[i])
            Raw_data_Columns_7.append(qNames_Combined[i])
            Raw_Data_7 = df[Raw_data_Columns_7]
        elif i==7:
            #Index_Raw_data_Columns_8.append(qCodes_Combined[i])
            #Raw_data_Columns_8.append(qCodes_Combined[i])
            Raw_data_Columns_8.append(qNames_Combined[i])
            Raw_Data_8 = df[Raw_data_Columns_8]
        elif i==8:
            #Index_Raw_data_Columns_9.append(qCodes_Combined[i])
            #Raw_data_Columns_9.append(qCodes_Combined[i])
            Raw_data_Columns_9.append(qNames_Combined[i])
            Raw_Data_9 = df[Raw_data_Columns_9]
        elif i==9:
            #Index_Raw_data_Columns_10.append(qCodes_Combined[i])
            #Raw_data_Columns_10.append(qCodes_Combined[i])
            Raw_data_Columns_10.append(qNames_Combined[i])
            Raw_Data_10 = df[Raw_data_Columns_10]

    #Raw_Data_1 = df[Raw_data_Columns_1]
    #Raw_Data_2 = df[Raw_data_Columns_2]
    #Raw_Data_3 = df[Raw_data_Columns_3]
    #Raw_Data_4 = df[Raw_data_Columns_4]
    #Raw_Data_5 = df[Raw_data_Columns_5]
    #Raw_Data_6 = df[Raw_data_Columns_6]
    #Raw_Data_7 = df[Raw_data_Columns_7]
    #Raw_Data_8 = df[Raw_data_Columns_8]
    #Raw_Data_9 = df[Raw_data_Columns_9]
    #Raw_Data_10 = df[Raw_data_Columns_10]

    
    
    

    #Final_Raw_Data.to_excel(r'C:\Users\Shivaji Choursiya\Desktop\OHI\PYTHONNNN\Test data\Testcvag_V6.xlsx', sheet_name='Raw data 1', index = False)

    
    writer.save()
    writer2.save()
    ProgressText = ProgressText + "\nInfluencer Report created and stored at Location: " + File_Save_Path
    ProgressLabel_F1['text'] = ProgressText
    exit()


In [4]:
def Main():
    
    def NominatedNames_Explode(Raw_Data_def ,Index_Raw_data_Columns_def,i):
        optedOutRespondents = Raw_Data_def.loc[Raw_Data_def[Optout_Combined[(int(i)*2)+1]] == '1',"sender_Name"]
        Raw_Data_def = Raw_Data_def[Raw_Data_def[Optout_Combined[int(i)*2]] != '1']
                
        #print(Index_Raw_data_Columns_def)
        #print(Raw_Data_def.columns)
                
        Raw_Data_def = (Raw_Data_def.set_index(Index_Raw_data_Columns_def).stack().str.split(',', expand=True).stack().unstack(-2).reset_index(-1, drop=True).reset_index())
                
        Raw_Data_def = Raw_Data_def[Raw_Data_def[qCodes_Combined[i]] != '']
        Raw_Data_def[qCodes_Combined[i]] = Raw_Data_def[qCodes_Combined[i]].astype(int)
        
        Respondent_id_Name = dict(zip(RespondentDataDF[influencerid], RespondentDataDF[influencerName]))
        Raw_Data_def['nominated_name'] = Raw_Data_def[qCodes_Combined[i]].map(Respondent_id_Name)
        
        for names in optedOutRespondents:
            Raw_Data_def = Raw_Data_def[Raw_Data_def['nominated_name'] != names]
            
        Final_Raw_Data = pd.merge(Raw_Data_def,
                                  RespondentDataDF_Label[Res_Col_Nominated],
                                  left_on=qCodes_Combined[i],
                                  right_on=influencerid,
                                  how='left')
        Final_Raw_Data = Final_Raw_Data.drop_duplicates()
        #Final_Raw_Data.columns = [str(x).lower() for x in Final_Raw_Data.columns]
        return Final_Raw_Data
    
    
    
    def Aggregated_Data(Final_Raw_Data_def,i):        
        #Operation On Aggregate Data
        Res_Col_Nominated.insert(0,'nominated_name')
        Aggregate_table = Final_Raw_Data_def[Res_Col_Nominated]
        Aggregate_table = Aggregate_table.fillna('')
        Aggregate_table_2 = Aggregate_table.groupby(Res_Col_Nominated).size().reset_index(name='Count')
        Res_Col_Nominated.remove(influencerid)
        
        Agg_ColList = np
        Agg_ColList = Aggregate_table_2.columns
        Agg_Col_Names =['nominated_name','Count']
        for colnames in Agg_ColList:
            if colnames.startswith('nominated_'):
                Agg_Col_Names.append(colnames)
        Aggregate_table_2 = Aggregate_table_2[Agg_Col_Names]
        #Aggregate_table_2.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
        Aggregate_table_2 = Aggregate_table_2.sort_values('Count', ascending=False)
        #Aggregate_table_2 = Aggregate_table_2.replace('',np.nan)
        return Aggregate_table_2
    
    
    
    class nofileException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
    
    class NoInfluencerNameFound(Exception):
        def __init__(self,msg):
            super().__init__(msg)
            
    class TextboxBlankException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
    class DuplicateNameException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
    
    
    
    ## Validating Inputs
    
    try:
        if RawLabel !="" and RespLabel !="" and SchemaLabel !="":
            ProgressText = "All files Imported"
            ProgressLabel['text'] = ProgressText
        else:
            raise nofileException("Please Provide entry in ALL the fields")
    
    except nofileException as e:
        ProgressLabel['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    except Exception:
        ProgressText = "+ Please Provide entry in ALL the fields"
        ProgressLabel['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    
    ## Importing File
    ###      > Survey data With Label
    ###      > Survey Schema
    ###      > Respondent Data
    
    
    influencerName = textbox1.get()
    influencerid = textbox3.get()
    File_Name = textbox2.get()
    
    df = pd.read_excel(RawLabel,dtype=object, keep_default_na = False)
    #SchemaDF = pd.read_csv(SchemaLabel,sep="\t")
    SchemaDF = pd.read_csv(SchemaLabel,sep="\t",dtype=object, na_filter = False)
    RespondentDataDF= pd.read_csv(RespLabel,sep="\t")
    
    
    try:
        if influencerName in RespondentDataDF.columns:
            ProgressText = ProgressText + "\nInfluencer Name " +influencerName + " is Present in Respondent file"
            ProgressLabel['text'] = ProgressText
        elif not influencerName.strip():
            raise TextboxBlankException("\nText Box cannot be left Blank")
            
        else:
            raise  NoInfluencerNameFound("\nInfluencer Name " + influencerName + " is NOT Present in Respondent file")
            
        
    except TextboxBlankException as e:
        ProgressText = ProgressText + "\nText Box Influencer Name cannot be left Blank"
        ProgressLabel['text'] = ProgressText
        m_box.showerror('Error', "Text Box Influencer Name cannot be left Blank")
        exit()
        return
        
            
    except NoInfluencerNameFound:
        ProgressText = ProgressText + "\nInfluencer Name " +influencerName + " is NOT Present in Respondent file"
        ProgressLabel['text'] = ProgressText
        m_box.showerror('Error', "\nInfluencer Name " +influencerName + " is NOT Present in Respondent file")
        exit()
        return
        
    except Exception:
        ProgressText = ProgressText + "\n + Influencer Name " +influencerName + " is NOT Present in Respondent file"
        ProgressLabel['text'] = ProgressText
        m_box.showerror('Error', "\n + Influencer Name " +influencerName + " is NOT Present in Respondent file")
        exit()
        return

        
    try:
        if not File_Name.strip():
            raise TextboxBlankException("\nText Box Survey Name cannot be left Blank")
        
    except TextboxBlankException:
        ProgressText = ProgressText + "\nText Box Survey Name cannot be left Blank"
        ProgressLabel['text'] = ProgressText
        m_box.showerror('Error', "Text Box Survey Name cannot be left Blank")
        exit()
        return
    except Exception:
        ProgressText = ProgressText + "\n + Text Box Survey Name cannot be left Blank"
        ProgressLabel['text'] = ProgressText
        m_box.showerror('Error', " + Text Box Survey Name cannot be left Blank")
        exit()
        return
    
    ##### Validating Names

    try:
        if RespondentDataDF[influencerName].duplicated().any():
            ProgressText = ProgressText + "\nInfluencer Name Column " +influencerName + " has duplicate names"
            ProgressLabel_F1['text'] = ProgressText
            raise DuplicateNameException("\nThere are duplicate Influencer Names in File")
    
    except DuplicateNameException as e:
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    except Exception:
        ProgressText = "+ There are duplicate Influencer Names in File"
        ProgressLabel_F1['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
        
        
    #### Converting Respondent data into Labelled data

    ResColList = np
    ResColList = RespondentDataDF.columns.str.replace('demo_','nominated_demo_')
    RespondentDataDF.columns = ResColList

    Res_Col_Names =['respid',influencerName, influencerid]
    Res_Col_Nominated =[influencerName, influencerid]
    for colnames in ResColList:
        if colnames.startswith('nominated'):
            Res_Col_Names.append(colnames)
            Res_Col_Nominated.append(colnames)

    RespondentDataDF_Label = RespondentDataDF.copy()
    User_Label = dict(zip(SchemaDF['Answer Code'], SchemaDF['Answer Label']))

    for colnames in Res_Col_Names:
        if colnames.startswith('nominated'):
            RespondentDataDF_Label[colnames] = RespondentDataDF[colnames].map(User_Label)



    ## Operations With RAW Survey Data

    RawColList = np
    RawColList = df.columns.str.replace('demo_','sender_demo_')
    df.columns = RawColList

    ###Inserting Sender Name
    
    df.insert(8,'sender_Name','')
    
    Respondent_Name = dict(zip(RespondentDataDF['respid'], RespondentDataDF[influencerName]))
    df['sender_Name'] = df['respid'].map(Respondent_Name)

    
    All_cols = df.columns
    Raw_data_Columns = ['responseid','respid','status','Live_Test','Ind_Global','sender_Name','qhidTemplateToUse','l']

    Optout_Combined = []
    qCodes_Combined = []
    #qNames_Combined = []
    for colnames in All_cols:
        if colnames.startswith('sender_demo_') and colnames.count('_rr_')==0:
            Raw_data_Columns.append(colnames)
        if colnames.startswith('Optout'):
            Optout_Combined.append(str(colnames))
        #if colnames.startswith('qNames'):
            #qNames_Combined.append(str(colnames))
        if colnames.startswith('qCodes'):
            qCodes_Combined.append(str(colnames))
            

    extra_cols = {'sender_demo_StraightLiners','sender_demo_Speeder','sender_demo_SL'}
    Raw_data_Columns_after = [rem for rem in Raw_data_Columns if rem not in extra_cols]

    Raw_data_Columns_1 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_1 = Raw_data_Columns_after.copy()

    Raw_data_Columns_2 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_2 = Raw_data_Columns_after.copy()

    Raw_data_Columns_3 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_3 = Raw_data_Columns_after.copy()

    Raw_data_Columns_4 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_4 = Raw_data_Columns_after.copy()

    Raw_data_Columns_5 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_5 = Raw_data_Columns_after.copy()

    Raw_data_Columns_6 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_6 = Raw_data_Columns_after.copy()

    Raw_data_Columns_7 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_7 = Raw_data_Columns_after.copy()

    Raw_data_Columns_8 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_8 = Raw_data_Columns_after.copy()

    Raw_data_Columns_9 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_9 = Raw_data_Columns_after.copy()

    Raw_data_Columns_10  = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_10 = Raw_data_Columns_after.copy()





    OptLength = len(Optout_Combined)
    #print(OptLength)
    for i in range(OptLength):
        if i<2:
            Raw_data_Columns_1.append(Optout_Combined[i])
            Index_Raw_data_Columns_1.append(Optout_Combined[i])
        elif i>=2 and i<4:
            Raw_data_Columns_2.append(Optout_Combined[i])
            Index_Raw_data_Columns_2.append(Optout_Combined[i])
        elif i>=4 and i<6:
            Raw_data_Columns_3.append(Optout_Combined[i])
            Index_Raw_data_Columns_3.append(Optout_Combined[i])
        elif i>=6 and i<8:
            Raw_data_Columns_4.append(Optout_Combined[i])
            Index_Raw_data_Columns_4.append(Optout_Combined[i])
        elif i>=8 and i<10:
            Raw_data_Columns_5.append(Optout_Combined[i])
            Index_Raw_data_Columns_5.append(Optout_Combined[i])
        elif i>=10 and i<12:
            Raw_data_Columns_6.append(Optout_Combined[i])
            Index_Raw_data_Columns_6.append(Optout_Combined[i])
        elif i>=12 and i<14:
            Raw_data_Columns_7.append(Optout_Combined[i])
            Index_Raw_data_Columns_7.append(Optout_Combined[i])
        elif i>=14 and i<16:
            Raw_data_Columns_8.append(Optout_Combined[i])
            Index_Raw_data_Columns_8.append(Optout_Combined[i])
        elif i>=16 and i<18:
            Raw_data_Columns_9.append(Optout_Combined[i])
            Index_Raw_data_Columns_9.append(Optout_Combined[i])
        elif i>=18 and i<20:
            Raw_data_Columns_10.append(Optout_Combined[i])
            Index_Raw_data_Columns_10.append(Optout_Combined[i])

    ###Creating Excel File
    File_Save_Path = RawLabel
    File_Save_Path = "/".join(File_Save_Path.split("/")[:-1])
    File_Save_Path = File_Save_Path+"/" +File_Name +"_Influencer_Report.xlsx"
    writer = pd.ExcelWriter(File_Save_Path, engine='xlsxwriter')
    
    File_Save_Path2 = RawLabel
    File_Save_Path2 = "/".join(File_Save_Path2.split("/")[:-1])
    File_Save_Path2 = File_Save_Path2+"/" +File_Name +"_Check_Blanks.xlsx"
    writer2 = pd.ExcelWriter(File_Save_Path2, engine='xlsxwriter')
    
    RespondentDataDF = RespondentDataDF[RespondentDataDF['Live_Test']==1]
    RespondentDataDF = RespondentDataDF.dropna(subset=[influencerid])
    RespondentDataDF[influencerid]=RespondentDataDF[influencerid].astype(int)
    
    
    
    
    
            
    qCodesLength = len(qCodes_Combined)
    
    for i in range(qCodesLength):
        if i==0:
            #Index_Raw_data_Columns_1.append(qCodes_Combined[i])
            Raw_data_Columns_1.append(qCodes_Combined[i])
            #Raw_data_Columns_1.append(qNames_Combined[i])
            Raw_Data_1 = df[Raw_data_Columns_1]
            
                
            if Raw_Data_1[qCodes_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_1 = NominatedNames_Explode(Raw_Data_1 ,Index_Raw_data_Columns_1,i)
                Aggregated_data_table_1 = Aggregated_Data(Final_Raw_Data_1,i)
                
                Final_Raw_Data_1 = Final_Raw_Data_1.drop([influencerid,influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1],qCodes_Combined[i]], axis=1)
                #Final_Raw_Data_1.rename(columns = {qCodes_Combined[i]:'nominated_name'}, inplace=True)
                
                
                
                Raw_data_Name_1 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_1 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_1.columns = [str(x).lower() for x in Final_Raw_Data_1.columns]
                Aggregated_data_table_1.columns = [str(x).lower() for x in Aggregated_data_table_1.columns]
                
                Final_Raw_Data_1.to_excel(writer, sheet_name=Raw_data_Name_1, index = False)
                Aggregated_data_table_1.to_excel(writer, sheet_name=Aggregate_sheet_Name_1, index = False)
                
                #Blank_Counts = Final_Raw_Data_1.isnull().sum().to_frame()
                Blank_Counts = countBlank(Final_Raw_Data_1)
                #Blank_Counts.reset_index(inplace=True)
                Blank_Counts.to_excel(writer2, sheet_name=Raw_data_Name_1, index = False)
                       

        elif i==1:
            #Index_Raw_data_Columns_2.append(qCodes_Combined[i])
            Raw_data_Columns_2.append(qCodes_Combined[i])
            #Raw_data_Columns_2.append(qCodes_Combined[i])
            Raw_Data_2 = df[Raw_data_Columns_2]
            
            if Raw_Data_2[qCodes_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_2 = NominatedNames_Explode(Raw_Data_2,Index_Raw_data_Columns_2,i)
                Aggregated_data_table_2 = Aggregated_Data(Final_Raw_Data_2,i)
                
                Final_Raw_Data_2 = Final_Raw_Data_2.drop([influencerid,influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1],qCodes_Combined[i]], axis=1)
                #Final_Raw_Data_2.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_2 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_2 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_2.columns = [str(x).lower() for x in Final_Raw_Data_2.columns]
                Aggregated_data_table_2.columns = [str(x).lower() for x in Aggregated_data_table_2.columns]
                
                Final_Raw_Data_2.to_excel(writer, sheet_name=Raw_data_Name_2, index = False)
                Aggregated_data_table_2.to_excel(writer, sheet_name=Aggregate_sheet_Name_2, index = False)
                
                #Blank_Counts2 = Final_Raw_Data_2.isnull().sum().to_frame()
                Blank_Counts2 = countBlank(Final_Raw_Data_2)
                #Blank_Counts2.reset_index(inplace=True)
                Blank_Counts2.to_excel(writer2, sheet_name=Raw_data_Name_2, index = False)
            
           
        elif i==2:
            #Index_Raw_data_Columns_3.append(qCodes_Combined[i])
            Raw_data_Columns_3.append(qCodes_Combined[i])
            #Raw_data_Columns_3.append(qNames_Combined[i])
            Raw_Data_3 = df[Raw_data_Columns_3]
            
            if Raw_Data_3[qCodes_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_3 = NominatedNames_Explode(Raw_Data_3,Index_Raw_data_Columns_3,i)
                Aggregated_data_table_3 = Aggregated_Data(Final_Raw_Data_3,i)
                
                Final_Raw_Data_3 = Final_Raw_Data_3.drop([influencerid,influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1],qCodes_Combined[i]], axis=1)
                #Final_Raw_Data_3.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_3 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_3 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_3.columns = [str(x).lower() for x in Final_Raw_Data_3.columns]
                Aggregated_data_table_3.columns = [str(x).lower() for x in Aggregated_data_table_3.columns]
                
                Final_Raw_Data_3.to_excel(writer, sheet_name=Raw_data_Name_3, index = False)
                Aggregated_data_table_3.to_excel(writer, sheet_name=Aggregate_sheet_Name_3, index = False)
                
                #Blank_Counts3 = Final_Raw_Data_3.isnull().sum().to_frame()
                Blank_Counts3 = countBlank(Final_Raw_Data_3)
                #Blank_Counts3.reset_index(inplace=True)
                Blank_Counts3.to_excel(writer2, sheet_name=Raw_data_Name_3, index = False)
            
            
        elif i==3:
            #Index_Raw_data_Columns_4.append(qCodes_Combined[i])
            Raw_data_Columns_4.append(qCodes_Combined[i])
            #Raw_data_Columns_4.append(qNames_Combined[i])
            Raw_Data_4 = df[Raw_data_Columns_4]
            
            if Raw_Data_4[qCodes_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_4 = NominatedNames_Explode(Raw_Data_4,Index_Raw_data_Columns_4,i)
                Aggregated_data_table_4 = Aggregated_Data(Final_Raw_Data_4,i)
                
                Final_Raw_Data_4 = Final_Raw_Data_4.drop([influencerid,influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1],qCodes_Combined[i]], axis=1)
                #Final_Raw_Data_4.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_4 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_4 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_4.columns = [str(x).lower() for x in Final_Raw_Data_4.columns]
                Aggregated_data_table_4.columns = [str(x).lower() for x in Aggregated_data_table_4.columns]
                
                Final_Raw_Data_4.to_excel(writer, sheet_name=Raw_data_Name_4, index = False)
                Aggregated_data_table_4.to_excel(writer, sheet_name=Aggregate_sheet_Name_4, index = False)
                
                #Blank_Counts4 = Final_Raw_Data_4.isnull().sum().to_frame()
                Blank_Counts4 = countBlank(Final_Raw_Data_4)
                #Blank_Counts4.reset_index(inplace=True)
                Blank_Counts4.to_excel(writer2, sheet_name=Raw_data_Name_4, index = False)
            
        elif i==4:
            #Index_Raw_data_Columns_5.append(qCodes_Combined[i])
            Raw_data_Columns_5.append(qCodes_Combined[i])
            #Raw_data_Columns_5.append(qNames_Combined[i])
            Raw_Data_5 = df[Raw_data_Columns_5]
            
            if Raw_Data_5[qCodes_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_5 = NominatedNames_Explode(Raw_Data_5,Index_Raw_data_Columns_5,i)
                Aggregated_data_table_5 = Aggregated_Data(Final_Raw_Data_5,i)
                
                Final_Raw_Data_5 = Final_Raw_Data_5.drop([influencerid,influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1],qCodes_Combined[i]], axis=1)
                #Final_Raw_Data_5.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_5 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_5 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_5.columns = [str(x).lower() for x in Final_Raw_Data_5.columns]
                Aggregated_data_table_5.columns = [str(x).lower() for x in Aggregated_data_table_5.columns]
                
                Final_Raw_Data_5.to_excel(writer, sheet_name=Raw_data_Name_5, index = False)
                Aggregated_data_table_5.to_excel(writer, sheet_name=Aggregate_sheet_Name_5, index = False)
                
                #Blank_Counts5 = Final_Raw_Data_5.isnull().sum().to_frame()
                Blank_Counts5 = countBlank(Final_Raw_Data_5)
                #Blank_Counts5.reset_index(inplace=True)
                Blank_Counts5.to_excel(writer2, sheet_name=Raw_data_Name_5, index = False)
            
            
        elif i==5:
            #Index_Raw_data_Columns_6.append(qCodes_Combined[i])
            Raw_data_Columns_6.append(qCodes_Combined[i])
            #Raw_data_Columns_6.append(qNames_Combined[i])
            Raw_Data_6 = df[Raw_data_Columns_6]
            
            if Raw_Data_6[qCodes_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_6 = NominatedNames_Explode(Raw_Data_6,Index_Raw_data_Columns_6,i)
                Aggregated_data_table_6 = Aggregated_Data(Final_Raw_Data_6,i)
                
                Final_Raw_Data_6 = Final_Raw_Data_6.drop([influencerid,influencerName,Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1],qCodes_Combined[i]], axis=1)
                #Final_Raw_Data_6.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_6 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_6 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_6.columns = [str(x).lower() for x in Final_Raw_Data_6.columns]
                Aggregated_data_table_6.columns = [str(x).lower() for x in Aggregated_data_table_6.columns]
                
                Final_Raw_Data_6.to_excel(writer, sheet_name=Raw_data_Name_6, index = False)
                Aggregated_data_table_6.to_excel(writer, sheet_name=Aggregate_sheet_Name_6, index = False)
                
                #Blank_Counts6 = Final_Raw_Data_6.isnull().sum().to_frame()
                Blank_Counts6 = countBlank(Final_Raw_Data_6)
                #Blank_Counts6.reset_index(inplace=True)
                Blank_Counts6.to_excel(writer2, sheet_name=Raw_data_Name_6, index = False)
            
        elif i==6:
            #Index_Raw_data_Columns_7.append(qCodes_Combined[i])
            Raw_data_Columns_7.append(qCodes_Combined[i])
            #Raw_data_Columns_7.append(qNames_Combined[i])
            Raw_Data_7 = df[Raw_data_Columns_7]
        elif i==7:
            #Index_Raw_data_Columns_8.append(qCodes_Combined[i])
            Raw_data_Columns_8.append(qCodes_Combined[i])
            #Raw_data_Columns_8.append(qNames_Combined[i])
            Raw_Data_8 = df[Raw_data_Columns_8]
        elif i==8:
            #Index_Raw_data_Columns_9.append(qCodes_Combined[i])
            Raw_data_Columns_9.append(qCodes_Combined[i])
            #Raw_data_Columns_9.append(qNames_Combined[i])
            Raw_Data_9 = df[Raw_data_Columns_9]
        elif i==9:
            #Index_Raw_data_Columns_10.append(qCodes_Combined[i])
            Raw_data_Columns_10.append(qCodes_Combined[i])
            #Raw_data_Columns_10.append(qNames_Combined[i])
            Raw_Data_10 = df[Raw_data_Columns_10]

    #Raw_Data_1 = df[Raw_data_Columns_1]
    #Raw_Data_2 = df[Raw_data_Columns_2]
    #Raw_Data_3 = df[Raw_data_Columns_3]
    #Raw_Data_4 = df[Raw_data_Columns_4]
    #Raw_Data_5 = df[Raw_data_Columns_5]
    #Raw_Data_6 = df[Raw_data_Columns_6]
    #Raw_Data_7 = df[Raw_data_Columns_7]
    #Raw_Data_8 = df[Raw_data_Columns_8]
    #Raw_Data_9 = df[Raw_data_Columns_9]
    #Raw_Data_10 = df[Raw_data_Columns_10]

    
    
    

    #Final_Raw_Data.to_excel(r'C:\Users\Shivaji Choursiya\Desktop\OHI\PYTHONNNN\Test data\Testcvag_V6.xlsx', sheet_name='Raw data 1', index = False)

    
    writer.save()
    writer2.save()
    ProgressText = ProgressText + "\nInfluencer Report created and stored at Location: " + File_Save_Path
    ProgressLabel['text'] = ProgressText
    exit()


In [5]:
def Main_F3():
        
    def NominatedNames_Explode(Raw_Data_def ,Index_Raw_data_Columns_def,i):
        #optedOutRespondents = Raw_Data_def.loc[Raw_Data_def[Optout_Combined[(int(i)*2)+1]] == 1,"sender_Name"]
        Raw_Data_def = Raw_Data_def[Raw_Data_def[Optout_Combined[int(i)*2]] != '1']
                
        #print(Index_Raw_data_Columns_def)
        #print(Raw_Data_def.columns)
                
        Raw_Data_def = (Raw_Data_def.set_index(Index_Raw_data_Columns_def).stack().str.split(' ✘', expand=True).stack().unstack(-2).reset_index(-1, drop=True).reset_index())
                
        Raw_Data_def = Raw_Data_def[Raw_Data_def[qNames_Combined[i]] != '']
        #for names in optedOutRespondents:
            #Raw_Data_def = Raw_Data_def[Raw_Data_def[qNames_Combined[i]] != names]
        
        Raw_Data_def.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
        
        Raw_Data_def = Raw_Data_def.drop_duplicates()
        
        #Final_Raw_Data.columns = [str(x).lower() for x in Final_Raw_Data.columns]
        return Raw_Data_def
    
    
    
    def Aggregated_Data(Raw_Data_def_Agg):        
        #Operation On Aggregate Data
        #Res_Col_Nominated.insert(0,'nominated_name')
        Aggregate_table= pd.DataFrame()
        Aggregate_table['nominated_name'] = Raw_Data_def_Agg['nominated_name']
        #Aggregate_table = Aggregate_table.fillna('')
        Aggregate_table_2 = Aggregate_table.groupby('nominated_name').size().reset_index(name='Count')
        Aggregate_table_2 = Aggregate_table_2.sort_values('Count', ascending=False)
        #Res_Col_Nominated.remove(influencerid)        
        return Aggregate_table_2
    
    
    
    class nofileException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
    
    class NoInfluencerNameFound(Exception):
        def __init__(self,msg):
            super().__init__(msg)
            
    class TextboxBlankException(Exception):
        def __init__(self,msg):
            super().__init__(msg)
    
    
    
    ## Validating Inputs
    
    try:
        if RawLabel_F3 !="":
            ProgressText = "All files Imported"
            ProgressLabel_F3['text'] = ProgressText
        else:
            raise nofileException("Please Provide entry in ALL the fields")
    
    except nofileException as e:
        ProgressLabel_F3['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    except Exception:
        ProgressText = "+ Please Provide entry in ALL the fields"
        ProgressLabel_F3['text'] = ProgressText
        m_box.showerror('Error', ProgressText)
        exit()
        return
    
    ## Importing File
    ###      > Survey data With Label
    
    File_Name = textbox2_F3.get()
    
    df = pd.read_excel(RawLabel_F3,dtype=object, keep_default_na = False)
    #SchemaDF = pd.read_csv(SchemaLabel,sep="\t")
        
    try:
        if not File_Name.strip():
            raise TextboxBlankException("\nText Box Survey Name cannot be left Blank")
        
    except TextboxBlankException:
        ProgressText = ProgressText + "\nText Box Survey Name cannot be left Blank"
        ProgressLabel_F3['text'] = ProgressText
        m_box.showerror('Error', "Text Box Survey Name cannot be left Blank")
        exit()
        return
    except Exception:
        ProgressText = ProgressText + "\n + Text Box Survey Name cannot be left Blank"
        ProgressLabel_F3['text'] = ProgressText
        m_box.showerror('Error', " + Text Box Survey Name cannot be left Blank")
        exit()
        return
        
        
        ## Operations With RAW Survey Data

    RawColList = np
    RawColList = df.columns.str.replace('demo_','sender_demo_')
    df.columns = RawColList

    ###Inserting Sender Name
    
    
    All_cols = df.columns
    Raw_data_Columns = ['responseid','respid','status','Live_Test','Ind_Global','qhidTemplateToUse','l']

    Optout_Combined = []
    #qCodes_Combined = []
    qNames_Combined = []
    for colnames in All_cols:
        if colnames.startswith('sender_demo_') and colnames.count('_rr_')==0:
            Raw_data_Columns.append(colnames)
        if colnames.startswith('Optout'):
            Optout_Combined.append(str(colnames))
        if colnames.startswith('qNames'):
            qNames_Combined.append(str(colnames))
        #if colnames.startswith('qCodes'):
            #qCodes_Combined.append(str(colnames))
            

    extra_cols = {'sender_demo_StraightLiners','sender_demo_Speeder','sender_demo_SL'}
    Raw_data_Columns_after = [rem for rem in Raw_data_Columns if rem not in extra_cols]

    Raw_data_Columns_1 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_1 = Raw_data_Columns_after.copy()

    Raw_data_Columns_2 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_2 = Raw_data_Columns_after.copy()

    Raw_data_Columns_3 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_3 = Raw_data_Columns_after.copy()

    Raw_data_Columns_4 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_4 = Raw_data_Columns_after.copy()

    Raw_data_Columns_5 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_5 = Raw_data_Columns_after.copy()

    Raw_data_Columns_6 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_6 = Raw_data_Columns_after.copy()

    Raw_data_Columns_7 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_7 = Raw_data_Columns_after.copy()

    Raw_data_Columns_8 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_8 = Raw_data_Columns_after.copy()

    Raw_data_Columns_9 = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_9 = Raw_data_Columns_after.copy()

    Raw_data_Columns_10  = Raw_data_Columns_after.copy()
    Index_Raw_data_Columns_10 = Raw_data_Columns_after.copy()





    OptLength = len(Optout_Combined)
    #print(OptLength)
    for i in range(OptLength):
        if i<2:
            Raw_data_Columns_1.append(Optout_Combined[i])
            Index_Raw_data_Columns_1.append(Optout_Combined[i])
        elif i>=2 and i<4:
            Raw_data_Columns_2.append(Optout_Combined[i])
            Index_Raw_data_Columns_2.append(Optout_Combined[i])
        elif i>=4 and i<6:
            Raw_data_Columns_3.append(Optout_Combined[i])
            Index_Raw_data_Columns_3.append(Optout_Combined[i])
        elif i>=6 and i<8:
            Raw_data_Columns_4.append(Optout_Combined[i])
            Index_Raw_data_Columns_4.append(Optout_Combined[i])
        elif i>=8 and i<10:
            Raw_data_Columns_5.append(Optout_Combined[i])
            Index_Raw_data_Columns_5.append(Optout_Combined[i])
        elif i>=10 and i<12:
            Raw_data_Columns_6.append(Optout_Combined[i])
            Index_Raw_data_Columns_6.append(Optout_Combined[i])
        elif i>=12 and i<14:
            Raw_data_Columns_7.append(Optout_Combined[i])
            Index_Raw_data_Columns_7.append(Optout_Combined[i])
        elif i>=14 and i<16:
            Raw_data_Columns_8.append(Optout_Combined[i])
            Index_Raw_data_Columns_8.append(Optout_Combined[i])
        elif i>=16 and i<18:
            Raw_data_Columns_9.append(Optout_Combined[i])
            Index_Raw_data_Columns_9.append(Optout_Combined[i])
        elif i>=18 and i<20:
            Raw_data_Columns_10.append(Optout_Combined[i])
            Index_Raw_data_Columns_10.append(Optout_Combined[i])

    ###Creating Excel File
    File_Save_Path = RawLabel_F3
    File_Save_Path = "/".join(File_Save_Path.split("/")[:-1])
    File_Save_Path = File_Save_Path+"/" +File_Name +"_Influencer_Report.xlsx"
    writer = pd.ExcelWriter(File_Save_Path, engine='xlsxwriter')
    
    File_Save_Path2 = RawLabel_F3
    File_Save_Path2 = "/".join(File_Save_Path2.split("/")[:-1])
    File_Save_Path2 = File_Save_Path2+"/" +File_Name +"_Check_Blanks.xlsx"
    writer2 = pd.ExcelWriter(File_Save_Path2, engine='xlsxwriter')
    
    qNamesLength = len(qNames_Combined)
        
    for i in range(qNamesLength):
        if i==0:
            #Index_Raw_data_Columns_1.append(qCodes_Combined[i])
            #Raw_data_Columns_1.append(qCodes_Combined[i])
            Raw_data_Columns_1.append(qNames_Combined[i])
            Raw_Data_1 = df[Raw_data_Columns_1]
            
            
            if Raw_Data_1[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_1 = NominatedNames_Explode(Raw_Data_1 ,Index_Raw_data_Columns_1,i)
                Aggregated_data_table_1 = Aggregated_Data(Final_Raw_Data_1)
                
                Final_Raw_Data_1 = Final_Raw_Data_1.drop([Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                #Final_Raw_Data_1.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_1 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_1 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_1.columns = [str(x).lower() for x in Final_Raw_Data_1.columns]
                Aggregated_data_table_1.columns = [str(x).lower() for x in Aggregated_data_table_1.columns]
                
                Final_Raw_Data_1.to_excel(writer, sheet_name=Raw_data_Name_1, index = False)
                Aggregated_data_table_1.to_excel(writer, sheet_name=Aggregate_sheet_Name_1, index = False)
                
                Blank_Counts1 = countBlank(Final_Raw_Data_1)
                #Blank_Counts1.reset_index(inplace=True)
                Blank_Counts1.to_excel(writer2, sheet_name=Raw_data_Name_1, index = False)

        elif i==1:
            #Index_Raw_data_Columns_2.append(qCodes_Combined[i])
            #Raw_data_Columns_2.append(qCodes_Combined[i])
            Raw_data_Columns_2.append(qNames_Combined[i])
            Raw_Data_2 = df[Raw_data_Columns_2]
            
            if Raw_Data_2[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_2 = NominatedNames_Explode(Raw_Data_1 ,Index_Raw_data_Columns_1,i)
                Aggregated_data_table_2 = Aggregated_Data(Final_Raw_Data_1)
                
                Final_Raw_Data_2 = Final_Raw_Data_2.drop([Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                #Final_Raw_Data_1.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_2 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_2 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_2.columns = [str(x).lower() for x in Final_Raw_Data_1.columns]
                Aggregated_data_table_2.columns = [str(x).lower() for x in Aggregated_data_table_1.columns]
                
                Final_Raw_Data_2.to_excel(writer, sheet_name=Raw_data_Name_2, index = False)
                Aggregated_data_table_2.to_excel(writer, sheet_name=Aggregate_sheet_Name_2, index = False)
                
                Blank_Counts2 = countBlank(Final_Raw_Data_2)
                #Blank_Counts2.reset_index(inplace=True)
                Blank_Counts2.to_excel(writer2, sheet_name=Raw_data_Name_2, index = False)
            
           
        elif i==2:
            #Index_Raw_data_Columns_3.append(qCodes_Combined[i])
            #Raw_data_Columns_3.append(qCodes_Combined[i])
            Raw_data_Columns_3.append(qNames_Combined[i])
            Raw_Data_3 = df[Raw_data_Columns_3]
            
            if Raw_Data_3[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_3 = NominatedNames_Explode(Raw_Data_3 ,Index_Raw_data_Columns_3,i)
                Aggregated_data_table_3 = Aggregated_Data(Final_Raw_Data_3)
                
                Final_Raw_Data_3 = Final_Raw_Data_3.drop([Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                #Final_Raw_Data_3.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_3 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_3 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_3.columns = [str(x).lower() for x in Final_Raw_Data_3.columns]
                Aggregated_data_table_3.columns = [str(x).lower() for x in Aggregated_data_table_3.columns]
                
                Final_Raw_Data_3.to_excel(writer, sheet_name=Raw_data_Name_3, index = False)
                Aggregated_data_table_3.to_excel(writer, sheet_name=Aggregate_sheet_Name_3, index = False)
                
                Blank_Counts3 = countBlank(Final_Raw_Data_3)
                #Blank_Counts3.reset_index(inplace=True)
                Blank_Counts3.to_excel(writer2, sheet_name=Raw_data_Name_3, index = False)
            
            
        elif i==3:
            #Index_Raw_data_Columns_4.append(qCodes_Combined[i])
            #Raw_data_Columns_4.append(qCodes_Combined[i])
            Raw_data_Columns_4.append(qNames_Combined[i])
            Raw_Data_4 = df[Raw_data_Columns_4]
            
            if Raw_Data_4[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_4 = NominatedNames_Explode(Raw_Data_4 ,Index_Raw_data_Columns_4,i)
                Aggregated_data_table_4 = Aggregated_Data(Final_Raw_Data_4)
                
                Final_Raw_Data_4 = Final_Raw_Data_4.drop([Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                #Final_Raw_Data_4.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_4 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_4 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_4.columns = [str(x).lower() for x in Final_Raw_Data_4.columns]
                Aggregated_data_table_4.columns = [str(x).lower() for x in Aggregated_data_table_4.columns]
                
                Final_Raw_Data_4.to_excel(writer, sheet_name=Raw_data_Name_4, index = False)
                Aggregated_data_table_4.to_excel(writer, sheet_name=Aggregate_sheet_Name_4, index = False)
                
                
                Blank_Counts4 = countBlank(Final_Raw_Data_4)
                #Blank_Counts4.reset_index(inplace=True)
                Blank_Counts4.to_excel(writer2, sheet_name=Raw_data_Name_4, index = False)
            
        elif i==4:
            #Index_Raw_data_Columns_5.append(qCodes_Combined[i])
            #Raw_data_Columns_5.append(qCodes_Combined[i])
            Raw_data_Columns_5.append(qNames_Combined[i])
            Raw_Data_5 = df[Raw_data_Columns_5]
            
            if Raw_Data_5[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_5 = NominatedNames_Explode(Raw_Data_5 ,Index_Raw_data_Columns_5,i)
                Aggregated_data_table_5 = Aggregated_Data(Final_Raw_Data_5)
                
                Final_Raw_Data_5 = Final_Raw_Data_5.drop([Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                #Final_Raw_Data_5.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_5 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_5 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_5.columns = [str(x).lower() for x in Final_Raw_Data_5.columns]
                Aggregated_data_table_5.columns = [str(x).lower() for x in Aggregated_data_table_5.columns]
                
                Final_Raw_Data_5.to_excel(writer, sheet_name=Raw_data_Name_5, index = False)
                Aggregated_data_table_5.to_excel(writer, sheet_name=Aggregate_sheet_Name_5, index = False)
                
                Blank_Counts5 = countBlank(Final_Raw_Data_5)
                #Blank_Counts5.reset_index(inplace=True)
                Blank_Counts5.to_excel(writer2, sheet_name=Raw_data_Name_5, index = False)
            
            
        elif i==5:
            #Index_Raw_data_Columns_6.append(qCodes_Combined[i])
            #Raw_data_Columns_6.append(qCodes_Combined[i])
            Raw_data_Columns_6.append(qNames_Combined[i])
            Raw_Data_6 = df[Raw_data_Columns_6]
            
            if Raw_Data_6[qNames_Combined[i]].replace('',np.nan).count()>0:
                
                Final_Raw_Data_6 = NominatedNames_Explode(Raw_Data_6 ,Index_Raw_data_Columns_6,i)
                Aggregated_data_table_6 = Aggregated_Data(Final_Raw_Data_6)
                
                Final_Raw_Data_6 = Final_Raw_Data_6.drop([Optout_Combined[int(i)*2],Optout_Combined[(int(i)*2)+1]], axis=1)
                #Final_Raw_Data_6.rename(columns = {qNames_Combined[i]:'nominated_name'}, inplace=True)
                
                Raw_data_Name_6 = 'Raw data_'+str(i+1)
                Aggregate_sheet_Name_6 ='Aggregated Data_'+str(+i+1)
                
                Final_Raw_Data_6.columns = [str(x).lower() for x in Final_Raw_Data_6.columns]
                Aggregated_data_table_6.columns = [str(x).lower() for x in Aggregated_data_table_6.columns]
                
                Final_Raw_Data_6.to_excel(writer, sheet_name=Raw_data_Name_6, index = False)
                Aggregated_data_table_6.to_excel(writer, sheet_name=Aggregate_sheet_Name_6, index = False)
                
                
                Blank_Counts6 = countBlank(Final_Raw_Data_6)
                #Blank_Counts6.reset_index(inplace=True)
                Blank_Counts6.to_excel(writer2, sheet_name=Raw_data_Name_6, index = False)
            
        elif i==6:
            #Index_Raw_data_Columns_7.append(qCodes_Combined[i])
            #Raw_data_Columns_7.append(qCodes_Combined[i])
            Raw_data_Columns_7.append(qNames_Combined[i])
            Raw_Data_7 = df[Raw_data_Columns_7]
        elif i==7:
            #Index_Raw_data_Columns_8.append(qCodes_Combined[i])
            #Raw_data_Columns_8.append(qCodes_Combined[i])
            Raw_data_Columns_8.append(qNames_Combined[i])
            Raw_Data_8 = df[Raw_data_Columns_8]
        elif i==8:
            #Index_Raw_data_Columns_9.append(qCodes_Combined[i])
            #Raw_data_Columns_9.append(qCodes_Combined[i])
            Raw_data_Columns_9.append(qNames_Combined[i])
            Raw_Data_9 = df[Raw_data_Columns_9]
        elif i==9:
            #Index_Raw_data_Columns_10.append(qCodes_Combined[i])
            #Raw_data_Columns_10.append(qCodes_Combined[i])
            Raw_data_Columns_10.append(qNames_Combined[i])
            Raw_Data_10 = df[Raw_data_Columns_10]

    #Raw_Data_1 = df[Raw_data_Columns_1]
    #Raw_Data_2 = df[Raw_data_Columns_2]
    #Raw_Data_3 = df[Raw_data_Columns_3]
    #Raw_Data_4 = df[Raw_data_Columns_4]
    #Raw_Data_5 = df[Raw_data_Columns_5]
    #Raw_Data_6 = df[Raw_data_Columns_6]
    #Raw_Data_7 = df[Raw_data_Columns_7]
    #Raw_Data_8 = df[Raw_data_Columns_8]
    #Raw_Data_9 = df[Raw_data_Columns_9]
    #Raw_Data_10 = df[Raw_data_Columns_10]

    
    
    

    #Final_Raw_Data.to_excel(r'C:\Users\Shivaji Choursiya\Desktop\OHI\PYTHONNNN\Test data\Testcvag_V6.xlsx', sheet_name='Raw data 1', index = False)

    
    writer.save()
    writer2.save()
    ProgressText = ProgressText + "\nInfluencer Report created and stored at Location: " + File_Save_Path
    ProgressLabel_F3['text'] = ProgressText
    exit()

In [6]:
HEIGHT = 600
WEIDTH = 900

root = tk.Tk()
root.title("Medusa")
root.iconbitmap("medusaicon.ico")

canvas = tk.Canvas(root, height=HEIGHT, width=WEIDTH)
canvas.pack()

boldFont = ('courier',10,'bold')
buttonFont = ('sans serif',9)

background_label = tk.Label(root, bg='#161338')
background_label.place(x=0, y=0, relwidth=1, relheight=1)

notebook_main = ttk.Notebook(root)
notebook_main.place(relx=0.5, rely=0.1, relwidth=0.75, relheight=0.9, anchor='n')

#frame_1 = Frame(notebook_main, width=500, height =500, bg="blue")
#frame_2 = Frame(notebook_main, width=500, height =500, bg="Red")

frame_1 = tk.Frame(root, bg='#292664', bd=5, highlightbackground="#3a29ff", highlightcolor="#3a29ff", highlightthickness=2)
frame_2 = tk.Frame(root, bg='#292664', bd=5, highlightbackground="#3a29ff", highlightcolor="#3a29ff", highlightthickness=2)
frame_3 = tk.Frame(root, bg='#292664', bd=5, highlightbackground="#3a29ff", highlightcolor="#3a29ff", highlightthickness=2)

frame_1.place(relx=0.5, rely=0.1,  relwidth=0.75, relheight=0.9, anchor='n')
frame_2.place(relx=0.5, rely=0.1,  relwidth=0.75, relheight=0.9, anchor='n')
frame_3.place(relx=0.5, rely=0.1,  relwidth=0.75, relheight=0.9, anchor='n')

notebook_main.add(frame_1,text="  USING INFLUENCER NAME  ")
notebook_main.add(frame_2,text="  USING INFLUENCER ID  ")
notebook_main.add(frame_3,text="  ONLY RAW DATA  ")

######################################
active_Tab = "#b05a1a"
inactive_Tab = "#FF6D00"

style = ttk.Style()

style.theme_create( "yummy", parent="alt", settings={
        "TNotebook": {"configure": {"tabmargins": [2, 5, 2, 0] } },
        "TNotebook.Tab": {
            "configure": {"padding": [5, 1], "background": active_Tab },
            "map":       {"background": [("selected", inactive_Tab)],
                          "expand": [("selected", [1, 1, 1, 0])] } } } )

style.theme_use("yummy")



#canvas = tk.Canvas(root, height=HEIGHT, width=WEIDTH)
#canvas.pack()

##############################################  FRAME 1   ######################################

def openRawData_F1():
    global RawLabel_F1
    root.Rawdatafilename_F1 = filedialog.askopenfilename(initialdir="/", title="Select Raw Data Fie", filetypes=(("Excel File",".xlsx"),("Excel files","*.xlsx")))
    RawDataDisp_F1 = root.Rawdatafilename_F1
    if RawDataDisp_F1!='':
        RawDataDisp_F1 = RawDataDisp_F1[RawDataDisp_F1.rindex('/')+1:]
    RawLabel_F1 = tk.Label(frame_1, text=RawDataDisp_F1, bg="#7a83ac", wraplength=400, font = boldFont, borderwidth=3, relief="solid")
    RawLabel_F1.place(relwidth=0.65, relheight=0.1)
    #excelfilepath = Label(root, text=root.Rawdatafilename_F1)
    RawLabel_F1 = root.Rawdatafilename_F1

def openRespData_F1():
    global RespLabel_F1
    root.RespDatafilename_F1 = filedialog.askopenfilename(initialdir="/", title="Select Respondent Data File", filetypes=(("Text File",".txt"),("Text files","*.txt")))
    RespDataDisp_F1 = root.RespDatafilename_F1
    if RespDataDisp_F1!='':
        RespDataDisp_F1 = RespDataDisp_F1[RespDataDisp_F1.rindex('/')+1:]    
    RespLabel_F1 = tk.Label(frame_1, text=RespDataDisp_F1, bg="#7a83ac", wraplength=400, font = boldFont, borderwidth=3, relief="solid")
    RespLabel_F1.place(rely=0.15,relwidth=0.65, relheight=0.1)
    RespLabel_F1 = root.RespDatafilename_F1
    
def openSchema_F1():
    global SchemaLabel_F1
    root.Schemafilename_F1 = filedialog.askopenfilename(initialdir="/", title="Select Survey Schema File", filetypes=(("Text File",".txt"),("Text files","*.txt")))
    SchemaDataDisp_F1 = root.Schemafilename_F1
    if SchemaDataDisp_F1!='':
        SchemaDataDisp_F1 = SchemaDataDisp_F1[SchemaDataDisp_F1.rindex('/')+1:]     
    SchemaLabel_F1 = tk.Label(frame_1, text=SchemaDataDisp_F1, bg="#7a83ac", wraplength=400, font = boldFont, borderwidth=3, relief="solid")
    SchemaLabel_F1.place(rely=0.3,relwidth=0.65, relheight=0.1)
    SchemaLabel_F1 = root.Schemafilename_F1


button_F1 =  tk.Button(frame_1, text = "RAW SURVEY DATA",command=openRawData_F1, bg='#FF6D00', activebackground="#b05a1a", fg='black', bd=8, font = buttonFont)
button_F1.place(relx=0.7, relwidth=0.3, relheight=0.1)


button2_F1 =  tk.Button(frame_1, text = "RESPONDENT DATA",command=openRespData_F1, bg='#FF6D00', activebackground="#b05a1a", fg='black',bd=8, font = buttonFont)
button2_F1.place(relx=0.7, rely=0.15, relwidth=0.3, relheight=0.1)


button3_F1 =  tk.Button(frame_1, text = "DEMOGRAPHICS SCHEMA", command=openSchema_F1, bg='#FF6D00', activebackground="#b05a1a", fg='black',bd=8, font = buttonFont)
button3_F1.place(relx=0.7, rely=0.3, relwidth=0.3, relheight=0.1)

textbox1_F1 = tk.Entry(frame_1, bg='white', borderwidth=3, relief="sunken")
textbox1_F1.place(relx=0.7, rely=0.45, relwidth=0.3, relheight=0.07)

LabelInfluencerName_F1 = tk.Label(frame_1, text='Insert Influencer Variable in Respondent file:', bg="#7a83ac", wraplength = 400, font = boldFont, borderwidth=3, relief="solid")
LabelInfluencerName_F1.place(rely=0.45,relwidth=0.65, relheight=0.07)

textbox2_F1 = tk.Entry(frame_1, bg='white', borderwidth=3, relief="sunken")
textbox2_F1.place(relx=0.7, rely=0.55, relwidth=0.3, relheight=0.07)

LabelSurveyName_F1 = tk.Label(frame_1, text='Insert Survey Name:', bg="#7a83ac", wraplength = 300, font = boldFont, borderwidth=3, relief="solid")
LabelSurveyName_F1.place(rely=0.55,relwidth=0.65, relheight=0.07)



Submit_F1 =  tk.Button(frame_1, text = "SUBMIT", command= lambda:Main_F1(), bg='#FF6D00', activebackground="#b05a1a", fg='black', bd=8, font = buttonFont)
Submit_F1.place(relx=0.45, rely=0.65, relwidth=0.15, relheight=0.1)

global ProglabelText_F1
ProglabelText_F1 = "Hope you are having a Great Day!"
ProgressLabel_F1 = tk.Label(frame_1, text=ProglabelText_F1, bg="#7a83ac", wraplength=500, borderwidth=5, relief="sunken")
ProgressLabel_F1.place(rely=0.75,relwidth=0.99, relheight=0.23)




##############################################  FRAME 2   ######################################

def openRawData():
    global RawLabel
    root.Rawdatafilename = filedialog.askopenfilename(initialdir="/", title="Select Raw Data Fie", filetypes=(("Excel File",".xlsx"),("Excel files","*.xlsx")))
    RawDataDisp = root.Rawdatafilename
    if RawDataDisp!='':
        RawDataDisp = RawDataDisp[RawDataDisp.rindex('/')+1:]
    Rawlabel = tk.Label(frame_2, text=RawDataDisp, bg="#7a83ac", wraplength=400, font = boldFont, borderwidth=3, relief="solid")
    Rawlabel.place(relwidth=0.65, relheight=0.08)
    #excelfilepath = Label(root, text=root.Rawdatafilename)
    RawLabel = root.Rawdatafilename

def openRespData():
    global RespLabel
    root.RespDatafilename = filedialog.askopenfilename(initialdir="/", title="Select Respondent Data File", filetypes=(("Text File",".txt"),("Text files","*.txt")))
    RespDataDisp = root.RespDatafilename
    if RespDataDisp!='':
        RespDataDisp = RespDataDisp[RespDataDisp.rindex('/')+1:]    
    RespLabel = tk.Label(frame_2, text=RespDataDisp, bg="#7a83ac", wraplength=400, font = boldFont, borderwidth=3, relief="solid")
    RespLabel.place(rely=0.12,relwidth=0.65, relheight=0.08)
    RespLabel = root.RespDatafilename
    
def openSchema():
    global SchemaLabel
    root.Schemafilename = filedialog.askopenfilename(initialdir="/", title="Select Survey Schema File", filetypes=(("Text File",".txt"),("Text files","*.txt")))
    SchemaDataDisp = root.Schemafilename
    if SchemaDataDisp!='':
        SchemaDataDisp = SchemaDataDisp[SchemaDataDisp.rindex('/')+1:]     
    SchemaLabel = tk.Label(frame_2, text=SchemaDataDisp, bg="#7a83ac", wraplength=400, font = boldFont, borderwidth=3, relief="solid")
    SchemaLabel.place(rely=0.24,relwidth=0.65, relheight=0.08)
    SchemaLabel = root.Schemafilename


button =  tk.Button(frame_2, text = "RAW SURVEY DATA",command=openRawData, bg='#FF6D00', activebackground="#b05a1a", fg='black', bd=8, font = buttonFont)
button.place(relx=0.7, relwidth=0.3, relheight=0.08)


button2 =  tk.Button(frame_2, text = "RESPONDENT DATA",command=openRespData, bg='#FF6D00', activebackground="#b05a1a", fg='black',bd=8, font = buttonFont)
button2.place(relx=0.7, rely=0.12, relwidth=0.3, relheight=0.08)


button3 =  tk.Button(frame_2, text = "DEMOGRAPHICS SCHEMA", command=openSchema, bg='#FF6D00', activebackground="#b05a1a", fg='black',bd=8, font = buttonFont)
button3.place(relx=0.7, rely=0.24, relwidth=0.3, relheight=0.08)


textbox3 = tk.Entry(frame_2, bg='white', borderwidth=3, relief="sunken")
textbox3.place(relx=0.7, rely=0.35, relwidth=0.3, relheight=0.07)

LabelInfluencerId = tk.Label(frame_2, text='Insert Influencer ID in Respondent file:', bg="#7a83ac", wraplength = 400, font = boldFont, borderwidth=3, relief="solid")
LabelInfluencerId.place(rely=0.35,relwidth=0.65, relheight=0.07)

textbox1 = tk.Entry(frame_2, bg='white', borderwidth=3, relief="sunken")
textbox1.place(relx=0.7, rely=0.46, relwidth=0.3, relheight=0.07)

LabelInfluencerName = tk.Label(frame_2, text='Insert Influencer Variable in Respondent file:', bg="#7a83ac", wraplength = 400, font = boldFont, borderwidth=3, relief="solid")
LabelInfluencerName.place(rely=0.46,relwidth=0.65, relheight=0.07)


textbox2 = tk.Entry(frame_2, bg='white', borderwidth=3, relief="sunken")
textbox2.place(relx=0.7, rely=0.57, relwidth=0.3, relheight=0.07)

LabelSurveyName = tk.Label(frame_2, text='Insert Survey Name:', bg="#7a83ac", wraplength = 300, font = boldFont, borderwidth=3, relief="solid")
LabelSurveyName.place(rely=0.57,relwidth=0.65, relheight=0.07)

Submit =  tk.Button(frame_2, text = "SUBMIT", command= lambda:Main(), bg='#FF6D00', activebackground="#b05a1a", fg='black', bd=8, font = buttonFont)
Submit.place(relx=0.45, rely=0.66, relwidth=0.15, relheight=0.08)

global ProglabelText
ProglabelText = "Hope you are having a Great Day!"
ProgressLabel = tk.Label(frame_2, text=ProglabelText, bg="#7a83ac", wraplength=500, borderwidth=5, relief="sunken")
ProgressLabel.place(rely=0.75,relwidth=0.99, relheight=0.23)

##############################################  FRAME 3   ######################################


def openRawData_F3():
    global RawLabel_F3
    root.Rawdatafilename_F3 = filedialog.askopenfilename(initialdir="/", title="Select Raw Data Fie", filetypes=(("Excel File",".xlsx"),("Excel files","*.xlsx")))
    RawDataDisp_F3 = root.Rawdatafilename_F3
    if RawDataDisp_F3!='':
        RawDataDisp_F3 = RawDataDisp_F3[RawDataDisp_F3.rindex('/')+1:]
    RawLabel_F3 = tk.Label(frame_3, text=RawDataDisp_F3, bg="#7a83ac", wraplength=400, font = boldFont, borderwidth=3, relief="solid")
    RawLabel_F3.place(rely=0.25, relwidth=0.65, relheight=0.1)
    #excelfilepath = Label(root, text=root.Rawdatafilename_F3)
    RawLabel_F3 = root.Rawdatafilename_F3


button_F3 =  tk.Button(frame_3, text = "RAW SURVEY DATA",command=openRawData_F3, bg='#FF6D00', activebackground="#b05a1a", fg='black', bd=8, font = buttonFont)
button_F3.place(relx=0.7, rely=0.25, relwidth=0.3, relheight=0.1)


textbox2_F3 = tk.Entry(frame_3, bg='white', borderwidth=3, relief="sunken")
textbox2_F3.place(relx=0.7, rely=0.40, relwidth=0.3, relheight=0.07)

LabelSurveyName_F3 = tk.Label(frame_3, text='Insert Survey Name:', bg="#7a83ac", wraplength = 300, font = boldFont, borderwidth=3, relief="solid")
LabelSurveyName_F3.place(rely=0.40,relwidth=0.65, relheight=0.07)

Submit_F3 =  tk.Button(frame_3, text = "SUBMIT", command= lambda:Main_F3(), bg='#FF6D00', activebackground="#b05a1a", fg='black', bd=8, font = buttonFont)
Submit_F3.place(relx=0.45, rely=0.55, relwidth=0.15, relheight=0.1)


global ProglabelText_F3
ProglabelText_F3 = "Hope you are having a Great Day!"
ProgressLabel_F3 = tk.Label(frame_3, text=ProglabelText_F3, bg="#7a83ac", wraplength=500, borderwidth=5, relief="sunken")
ProgressLabel_F3.place(rely=0.75,relwidth=0.99, relheight=0.23)


root.mainloop()